# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [35]:
# Details Student 1:
##Matan Mizrahi

# Details Student 2:
##209413004

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [36]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [37]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [38]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
# !python -m wn download omw-he:1.4

In [39]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [40]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer


In [41]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [42]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [43]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [44]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [45]:
# YOUR CODE HERE
def preprocess_text(text):
    tokens = ht.tokenize(text, with_whitespaces=True)
    normalized_tokens = [token for grp, token, _, _ in tokens if grp == 'HEBREW']
    return " ".join(normalized_tokens)

In [46]:
df_train['processed_story'] = df_train['story'].apply(preprocess_text)
df_test['processed_story'] = df_test['story'].apply(preprocess_text)

In [47]:
# Add as many code cells as you need
# YOUR CODE HERE
def oversample_female_class(df_train):
    num_female_samples = len(df_train[df_train['gender'] == 'f'])
    num_male_samples = len(df_train[df_train['gender'] == 'm'])
    oversample_factor = num_male_samples // num_female_samples
    df_female_training = df_train[df_train['gender'] == 'f']
    df_male_training = df_train[df_train['gender'] == 'm']
    df_female_oversampled = df_female_training.sample(oversample_factor * num_female_samples, replace=True)
    return pd.concat([df_male_training, df_female_oversampled])

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split
def train_and_evaluate_best_model(X, y):
    models = {
        "MultinomialNB": MultinomialNB(),
        "LogisticRegression": LogisticRegression(),
        "SVC": SVC(kernel='linear'),
        "RandomForest": RandomForestClassifier(),
    }
    
    best_model = None
    best_score = 0.0
    
    for model_name, model in models.items():
        print(f"Evaluating {model_name}...")
        
        # Cross-validation
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='f1_macro')
        average_f1 = cv_scores.mean()#(f1_male+f1_female)/2
        print(f"Average F1 score on cross-validation for {model_name}: {average_f1}")
        
        # Model training
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        model.fit(X_train, y_train)
        y_pred_val = model.predict(X_val)
        
        f1_score_val = f1_score(y_val, y_pred_val, average='macro')
        print(f"F1 score on validation set for {model_name}: {f1_score_val}")
        
        if f1_score_val > best_score:
            best_score = f1_score_val
            best_model = model_name
            
    print(f"The best model is {best_model} with a score of {best_score}")
    return models[best_model]


In [49]:
def save_predictions_to_csv(predictions, df_test, filename='classification_results.csv'):
    df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': predictions})
    df_predicted.to_csv(filename, index=False)


In [50]:
def display_predictions(y_pred_test):
    initialFivepred= y_pred_test[:5]
    LastFivePred = y_pred_test[-5:]
    print("first five preds are:",initialFivepred)
    print("last Five preds are:",LastFivePred)

In [51]:
df_train['processed_story'] = df_train['story'].apply(preprocess_text)
df_test['processed_story'] = df_test['story'].apply(preprocess_text)


In [52]:
df_train_oversampled = oversample_female_class(df_train)


In [53]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_oversampled['processed_story'])
y = df_train_oversampled['gender']

In [54]:
# choosing the best model 
best_model = train_and_evaluate_best_model(X, y)


Evaluating MultinomialNB...
Average F1 score on cross-validation for MultinomialNB: 0.9160581335646093
F1 score on validation set for MultinomialNB: 0.8198198198198198
Evaluating LogisticRegression...
Average F1 score on cross-validation for LogisticRegression: 0.926205827253295
F1 score on validation set for LogisticRegression: 0.9209848627598769
Evaluating SVC...
Average F1 score on cross-validation for SVC: 0.9428425572280513
F1 score on validation set for SVC: 0.9444861215303826
Evaluating RandomForest...
Average F1 score on cross-validation for RandomForest: 0.9473783790521143
F1 score on validation set for RandomForest: 0.9630984042553192
The best model is RandomForest with a score of 0.9630984042553192


In [55]:
X_test = vectorizer.transform(df_test['processed_story'])
y_pred_test = best_model.predict(X_test)

In [56]:
display_predictions(y_pred_test)

first five preds are: ['m' 'm' 'f' 'm' 'm']
last Five preds are: ['m' 'm' 'm' 'm' 'm']


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following: